In [ ]:
# Import the packages that we will use
import hvplot.xarray
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    import met4450
except ImportError:
    print('Uh oh!!')
    print('You must have met4450.py in the same directory as this file or somewhere else in your python path')

In [ ]:
# Connect to Amazon Web Services public datastores
import s3fs
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
How to use find_goes_dataset
The function find_goes_filename within the met4450 package makes is easy to find GOES data on AWS. The keyword arguments are

satellite - defines which GOES satellite to use. Possible values are
16, '16', 'G16', 'GOES16', 'GOES-16', 'GOES-EAST', 'EAST' = GOES-16
17, '17', 'G17', 'GOES17', 'GOES-17' = GOES-17
18, '18', 'G18', 'GOES18', 'GOES-18', 'GOES-WEST', 'WEST' = GOES-18
product - defines which ABI product to use. Typically use 'ABI-L2-MCMIP', which is the cloud and moisture imagery for all bands in one file, all at the same resolution (2 km). See link for a full list of GOES-16, 17, 18 products.
domain - defines the ABI field of view. Possible values are
F = Full disk
C = CONUS
M1 or M2 = Mesoscale sector 1 or 2
datetime - defines the date and time of interest. Data for the nearest matching time will be returned. Use the format 'YYYY-MM-DD HH:MM'. (Timestamp and datetime objects are also OK, if you know what they are.) The datetime must be after the following operational start dates
2017-03-01 for GOES-16
2018-08-29 for GOES-17
2022-07-28 for GOES-18
The function will return the path to the data nearest in time to the datetime that you provide.

In [ ]:
# Find file for a date, time, product and domain of intrest
fileinfo = met4450.find_goes_dataset(satellite=16,
                        product='ABI-L2-MCMIP',
                        domain='C',
                        datetime='2023-02-14 19:00')

# Print the name of the file that we will read
print( 'Reading file: https://{:s}.s3.amazonaws.com/{:s}'.format( *fileinfo.file[0].split('/',1) ) )
                                           
# Load the data from AWS
goes = xr.open_dataset( fs.open( fileinfo.file[0] ) )

In [ ]:
The variable goes is a dataset that contains all contents of the netCDF file. The dataset contains many variables within it. For example, goes['x'] is the variable that contains the x coordinate of the data and goes['y'] is the y coordinate variable. Imagery from the 16 ABI bands are contained in the variables CMI_Cxx, where xx is a number 01-16, so goes['CMI_C01'] is the variable containing data from ABI band 1. See this link for a summary of the wavelengths corresponding to each band.

The following cell displays a summary of the dataset, from which you can explore all of the variables that it contains.

In [ ]:
# Display summary of dataset
# Explore the output
goes

In [ ]:
# Display band 3 (0.86µm) reflectance
goes['CMI_C03'].plot.imshow()

In [ ]:
# Repeat as an interactive plot
goes['CMI_C03'].hvplot('x','y',data_aspect=1)

In [ ]:
Explore
Experiment with the commands above. For example, try some different ABI bands, a different date and time, a different domain, or a different GOES satellite.

True color images
To create a true color image (aka natural color), we must define the brightness of red, blue, and green components. ABI has blue (band 1) and red (band 2) bands. We will treat band 3 (0.86 µm) as green because, as we have seen, green vegetation also reflects near IR light.

In [ ]:
# Define red, blue, and green colors for a natural color image
goes['R'] = goes['CMI_C02']
goes['G'] = goes['CMI_C03']
goes['B'] = goes['CMI_C01']

# Combine RGB into one data array
goes['RGB'] = goes[['R','G','B']].to_array()

# Restrict to range [0,1]
goes['RGB'] = np.clip( goes['RGB'], 0, 1 )